# AlexNet

In [1]:
import tensorflow as tf
import DL_utils as dl
import datetime
import numpy as np
import os
import h5py
import math
from tqdm import tqdm_notebook, tnrange

/home/aditya/Documents/Projects/atgm_vision_module/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.4.1'

In [3]:
def create_placeholders(n_H, n_W, n_C, n_y):
    '''
    Function to create placeholder for tensorflow session
    
    Args:
    n_H = height of the image
    n_W = width of image
    n_C = number of channels
    n_y = number of output features
    
    returns:
    X,Y
    '''
    X = tf.placeholder(tf.float32, shape = (None, n_H, n_W, n_C))
    Y = tf.placeholder(tf.float32, shape = (None, n_y))
    
    return X, Y

In [4]:
parameters = dl.initialize_weights('AlexNet.xml')

[11, 11, 3, 96] W01 b01
[5, 5, 96, 256] W02 b02
[3, 3, 256, 384] W03 b03
[3, 3, 384, 384] W04 b04
[3, 3, 384, 256] W05 b05


In [5]:
def AN_forward_propagation(X, parameters):
    '''
    Forward propagation for AlexNet without normalization
    
    '''
    # 1st conv
    A1 = dl.conv_layer(X,parameters['W01'],parameters['b01'], [1,4,4,1], padding='VALID', name='1')
    # 1st pool
    P1 = dl.max_pool(A1, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='1')
    # 2nd conv
    A2 = dl.conv_layer(P1,parameters['W02'],parameters['b02'], strides = [1,1,1,1], padding = 'SAME', name='2')
    #2nd pool
    P2 = dl.max_pool(A2, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='2')
    # 3rd conv
    A3 = dl.conv_layer(P2,parameters['W03'], parameters['b03'], strides=[1,1,1,1], padding='SAME', name='3')
    # 4th conv
    A4 = dl.conv_layer(A3,parameters['W04'], parameters['b04'], strides=[1,1,1,1], padding = 'SAME', name='4')
    # 5th conv
    A5 = dl.conv_layer(A4,parameters['W05'], parameters['b05'], strides=[1,1,1,1], padding='SAME', name='5')
    # 3rd pool
    P3 = dl.max_pool(A5, kernel=[1,3,3,1], strides =[1,2,2,1], padding = 'VALID', name='3')
    # Flattening the last pooling layer
    P3 = tf.contrib.layers.flatten(P3)
    # FC1 - 4096 neurons
    F1 = dl.fc_layer(P3, 4096, activation_fn=None, name='1')
    # FC2 - 4096 neurons
    F2 = dl.fc_layer(F1, 4096, activation_fn=None, name='2')
    # FC3 - 1000 neurons reping different classes - may be modified for other models
    F3 = dl.fc_layer(F2, 1000, activation_fn=None, name='3')
    
    return F3

In [6]:
def compute_AN_cost(Z, Y, name=None):
    '''
    Computes cost 
    
    Args:
    Z8 -- Logits/Linear output from the last fully connected layer
    Y -- labels corresponding to each example in the batch
    
    Returns:
    cost -- cost of the epoch
    '''
    with tf.name_scope("cost"):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))
        tf.summary.scalar("cost", cost)
        return cost

In [7]:
def model(xml_path, LOGDIR, MODELDIR, test_path, learning_rate = 0.09, num_epochs = 150, 
          minibatch_size = 64, print_cost = True):
    
    # restting the default graph
    tf.reset_default_graph()
    
    # retrieve image shapes
    #(m, n_Htr, n_Wtr, n_Ctr) = X_train.shape
    #n_Y = Y_train.shape
    
    # global variables
    costs = []
    
    # randomizer
    seed = int(np.random.randint(1,100,1))
    
    # creating placeholders 
    X, Y = create_placeholders(224,224,3,1000)
    
    # initializing parameters
    parameters = dl.initialize_weights(xml_path)
    
    # forward prop
    F3 = AN_forward_propagation(X, parameters)
    
    # compute cost
    cost = compute_AN_cost(F3, Y)
        
    # compute accuracy
    train_accuracy = dl.accuracy(F3, Y)
    test_accuracy = dl.accuracy(F3, Y, "test")
    
    # select the optimizer
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # initialize global variables
    init = tf.global_variables_initializer()
    
    # initialize the session
    with tf.Session() as sess:
        
        # run the initialization for the session
        sess.run(init)
        
        # initializing summaries and file writers
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter(LOGDIR)
        writer.add_graph(sess.graph)
        saver = tf.train.Saver()
        
        # for loop for epoch/iterations
        for epoch in tnrange(num_epochs):
            
            # maintain the cost through an epoch
            epoch_cost = 0
            
            # randomizer
            seed += 1
            
            # path to training folder
            PATH = '/home/aditya/Documents/Projects/atgm_vision_module/CNN-Implementations/data/h5_FILES_ImageNet/training_folder/'
            
            # set up the data
            h5_files = os.listdir(PATH)
            
            # for loop to iterate through the h5 files
            for file in tqdm_notebook(h5_files):
                
                # open the h5 file to form tensor
                with h5py.File(PATH+file, mode = 'r') as h5_file:
                    
                    # extract features and labels
                    X_train = np.asarray(h5_file['X_train'])
                    Y_train = np.asarray(h5_file['Y_train'])
                    
                    # number of examples
                    (m, n_Htr, n_Wtr, n_Ctr) = h5_file['X_train'].shape
                    
                    # ??? - REASON WHY
                    num_minibatches = int(m/minibatch_size)

                    # generate minibatches
                    minibatches = dl.random_mini_batches(X_train, Y_train, minibatch_size, seed)

                    # iterate through the minibatches
                    for minibatch in minibatches:
                        
                        # procure minibatches
                        (minibatch_X, minibatch_Y) = minibatch
                        # optimize for cost, 
                        _ , minibatch_cost = sess.run([train_step, cost], feed_dict={X: minibatch_X, Y: tf.one_hot(minibatch_Y, 1000).eval()})
                        # cumulative minibatch cost
                        epoch_cost += minibatch_cost/num_minibatches
                        
                        # training accuracy
                        training = sess.run(train_accuracy, feed_dict={X: minibatch_X, Y: tf.one_hot(minibatch_Y, 1000).eval()})
                        
                    # calculate the accuracy
                    if epoch % 5  == 0 and epoch != 0:

                        # testing accuracy
                        with h5py.File(test_path, mode='r') as h5_file_test:
                            number_of_test_images = h5_file_test['Y_val'].shape[0]
                            k = 0
                            num_of_test_batches = number_of_test_images/minibatch_size
                            tot_testing = 0
                            while (k*minibatch_size) < number_of_test_images:
                                X_test = np.asarray(h5_file_test['X_val'][k*minibatch_size:k*minibatch_size+minibatch_size,:,:,:])
                                Y_test = np.asarray(h5_file_test['Y_val'][k*minibatch_size:k*minibatch_size+minibatch_size])
                                testing = sess.run(test_accuracy, feed_dict={X: X_test, Y: tf.one_hot(Y_test, 1000).eval()})
                                tot_testing += testing/num_of_test_batches
                                k+=1

                        # print the accuracy
                        print("current training accuracy : ", training)
                        print("current test accuracy : ", tot_testing)
                        
                        # save summaries and models
                        s = sess.run(merged_summary, feed_dict = {X: X_train, Y: Y_train})
                        writer.add_summary(s, epoch)
                        save_path = saver.save(sess, MODELDIR)
            
            # Print the cost after every 5 epochs
            if print_cost == True and epoch % 1 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                # costs.append(epoch_cost)
                
    return parameters

In [ ]:
test_path = "/home/aditya/Documents/Projects/atgm_vision_module/CNN-Implementations/data/h5_FILES_ImageNet/validation_folder/val.hdf5"
xml_path = 'AlexNet.xml'
LOGDIR = '/home/aditya/Documents/Projects/atgm_vision_module/training_logs/tensorboard_logs'
now = datetime.datetime.now()
MODELDIR = '/home/aditya/Documents/Projects/atgm_vision_module/training_logs/model_ckpts/'+str(now.isoformat())+'.ckpt'
model(xml_path, LOGDIR, MODELDIR, test_path)

[11, 11, 3, 96] W01 b01
[5, 5, 96, 256] W02 b02
[3, 3, 256, 384] W03 b03
[3, 3, 384, 384] W04 b04
[3, 3, 384, 256] W05 b05


HBox(children=(IntProgress(value=0, max=150), HTML(value='')))

HBox(children=(IntProgress(value=0, max=651), HTML(value='')))